# Detecting COVID-19 with Chest X Ray using PyTorch
Image classification of Chest X Rays in one of three classes: Normal, Viral Pneumonia, COVID-19

Notebook created for the guided project Detecting COVID-19 with Chest X Ray using PyTorch on Coursera

Dataset from COVID-19 Radiography Dataset on Kaggle

# Import Libraries

In [36]:
%matplotlib inline

import os
import shutil
import random
import torch
import torchvision
import numpy as np

from PIL import Image
from matplotlib import pyplot as plt

torch.manual_seed(0)

print('Using PyTorch version', torch.__version__)

Using PyTorch version 1.4.0


# Preparing Training and Test Sets


In [37]:
class_names = ['normal', 'viral', 'covid']
root_dir = 'data/COVID-19 Radiography Database'
source_dirs = ['Normal', 'Viral Pneumonia', 'COVID']

if os.path.isdir(os.path.join(root_dir, source_dirs[1])):
    os.mkdir(os.path.join(root_dir, 'test'))

    for i, d in enumerate(source_dirs):
        os.rename(os.path.join(root_dir, d), os.path.join(root_dir, class_names[i]))

    for c in class_names:
        os.mkdir(os.path.join(root_dir, 'test', c))

    for c in class_names:
        images = [x for x in os.listdir(os.path.join(root_dir, c)) if x.lower().endswith('png')]
        selected_images = random.sample(images, 30)
        for image in selected_images:
            source_path = os.path.join(root_dir, c, image)
            target_path = os.path.join(root_dir, 'test', c, image)
            shutil.move(source_path, target_path)

# Creating custom Dataset

In [38]:
class ChestXRayDataset(torch.utils.data.Dataset):
    def __init__(self, image_dirs, transform):
        def get_images(class_name):
            images = [x for x in os.listdir(image_dirs[class_name]) if x.lower().endswith('png')]
            print(f'Found {len(images)} {class_name} examples')
            return images
        self.images = {}
        self.class_names = ['normal', 'viral', 'covid']
        
        for c in self.class_names:
            self.images[c] = get_images(c)
        
        self.image_dirs = image_dirs
        self.transorm = transform

    def __len__(self):
        return sum([len(self.images[c]) for c in self.class_names])
        
    def __getitem__(self, index):
        class_name = randdom.choice(self.class_names)
        index = index % len(self.images[class_name])
        image_name = self.images[class_name][index]
        image_path = os.path.join(self.image_dirs[class_name],image_name)
        image = Image.open(image_path).convert('RGB')
        return self.transform(image), self.class_names.index(class_name)



# Image Transformations

In [39]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# PrepareData Loaders

In [40]:
print("Train Dataset")
train_dirs = {
    'normal': '../data/COVID-19 Radiography Database/normal',
    'viral': '../data/COVID-19 Radiography Database/viral',
    'covid': '../data/COVID-19 Radiography Database/covid'
}

train_dataset = ChestXRayDataset(train_dirs, train_transform)

print("\nTest Dataset")
test_dirs = {
    'normal': '../data/COVID-19 Radiography Database/test/normal',
    'viral': '../data/COVID-19 Radiography Database/test/viral',
    'covid': '../data/COVID-19 Radiography Database/test/covid'
}
test_dataset = ChestXRayDataset(test_dirs, test_transform)

Train Dataset
Found 10162 normal examples
Found 1315 viral examples
Found 3586 covid examples

Test Dataset
Found 30 normal examples
Found 30 viral examples
Found 30 covid examples


In [41]:
batch_size = 6
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [42]:
print("Number of training batches ",len(train_data_loader))
print("Number of test batches ",len(test_data_loader))

Number of training batches  2511
Number of test batches  15
